In [ ]:
import yt_dlp
import json
import re
from datetime import datetime, timedelta

channel_url = 'YOUR_CHANNEL_URL_HERE'

ydl_opts = {
    'quiet': True,
    'extract_flat': 'in_playlist',
    'skip_download': True,
    'force_generic_extractor': True
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    result = ydl.extract_info(channel_url, download=False)

videos = []
for entry in result['entries']:
    video = {
        'url': entry['url'],
        'title': entry['title'],
        'upload_date': entry['upload_date']
    }
    videos.append(video)

print(json.dumps(videos, indent=4))


In [ ]:
import os

chat_transcripts = {}

for video in videos:
    video_url = video['url']
    video_id = video_url.split('=')[-1]
    chat_file = f'{video_id}.json'

    ydl_opts = {
        'quiet': True,
        'skip_download': True,
        'writesubtitles': True,
        'subtitlesformat': 'json3',
        'subtitleslangs': ['live_chat'],
        'outtmpl': chat_file
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

    if os.path.exists(chat_file):
        with open(chat_file, 'r', encoding='utf-8') as f:
            chat_transcripts[video_id] = json.load(f)

print(chat_transcripts)


In [ ]:
burp_patterns = re.compile(r'\b(10/10|8/10|11/10)\b')

burp_timestamps = {}

for video_id, transcript in chat_transcripts.items():
    burp_timestamps[video_id] = []
    for event in transcript['events']:
        if 'replayChatItemAction' in event:
            message = event['replayChatItemAction']['actions'][0]['addChatItemAction']['item']['liveChatTextMessageRenderer']['message']['runs'][0]['text']
            if burp_patterns.search(message):
                timestamp = event['replayChatItemAction']['videoOffsetTimeMsec']
                burp_timestamps[video_id].append(timestamp)

print(burp_timestamps)


In [ ]:
def detect_burps(timestamps, threshold=2, interval=10):
    detected_burps = []
    for i in range(len(timestamps)):
        count = 1
        for j in range(i + 1, len(timestamps)):
            if (timestamps[j] - timestamps[i]) <= interval * 1000:
                count += 1
            else:
                break
        if count >= threshold:
            detected_burps.append(timestamps[i])
    return detected_burps

for video_id, timestamps in burp_timestamps.items():
    burp_timestamps[video_id] = detect_burps(timestamps)

print(burp_timestamps)


In [ ]:
clickable_urls = {}

for video in videos:
    video_id = video['url'].split('=')[-1]
    clickable_urls[video_id] = []
    for timestamp in burp_timestamps[video_id]:
        clickable_url = f'https://www.youtube.com/watch?v={video_id}&t={int(timestamp / 1000)}s'
        clickable_urls[video_id].append(clickable_url)

print(clickable_urls)


In [ ]:
with open('burp_timestamps.json', 'w', encoding='utf-8') as f:
    json.dump(clickable_urls, f, indent=4)

print('Timestamps saved to burp_timestamps.json')
